## Create Model Components
Creat the distance bands and normalised distance from glacier edge tif files used in inversion

In [1]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator, interp1d, RectBivariateSpline
# import matplotlib as mpl
import matplotlib.pyplot as plt
import rioxarray
# import fiona
# import rasterio.mask
import pandas as pd
# import verde as vd
import geopandas as gpd
import time
# from shapely.geometry import *
# import rasterio as rs

import numpy as np
import xarray as xr
# import pyvista as pv
# import harmonica as hm
from shapely import geometry

#from discretize.utils import mkvc


In [2]:
#read in the data measurement locations
profile4 = pd.read_csv('Data/prof4_meas.csv')
profile7a = pd.read_csv('Data/prof7a_meas.csv')
longa = pd.read_csv('Data/longa_meas.csv')

In [7]:
def open_raster(file): #create a function to read in the dem in tif format and convert it to an xarray which is used by harmonica
    #note function returns a dataArray and a Dataset which are different things - we use the dataArray - and also an array of x,y,z values
    geotiff_da = rioxarray.open_rasterio(file)
    geotiff_ds = geotiff_da.to_dataset('band') # Covert our xarray.DataArray into a xarray.Dataset
    geotiff_ds = geotiff_ds.rename({1: 'topo'}) # Rename the variable to a more useful name in the dataset

    geo_tiff_topo = geotiff_da[0] # in the dataArray select just the first variable which is the topography

    [x_topo, y_topo] = np.meshgrid(geo_tiff_topo.x.values, geo_tiff_topo.y.values) # the x and y values are read in as a single array for each and we want to create a grid of all values
    z_topo = geo_tiff_topo.values # defining the z component as the topography

    topo_xyz = np.c_[x_topo.ravel(), y_topo.ravel(), z_topo.ravel()] # we combine the x,y and z information into one array to be returned as a standard array

    topo_xr = xr.DataArray(geo_tiff_topo.values, # create the DataArray
    coords={'y': y_topo[:,0],'x': x_topo[0,:]}, 
    dims=["y", "x"])

    return topo_xr #returns the dataset, numpy array and dataArray

surf_xr = open_raster('Data/arctic_dem_plus_10m_nad83.tif')
bed_xr = open_raster('Data/Farinotti_2019/taku_plus_bed_nad83.tif')
bed_xr = bed_xr.reindex(y=bed_xr.y[::-1]) #invert y axis so it is negative to positive

model_shape = gpd.read_file('Model_components/model_domain_nad83.shp') #load in shapefile of glacier shape for the area of measurements
model_geom = geometry.shape(model_shape['geometry'].values[0])

dist_xr = open_raster('Model_components/taku_edge_dist_nad83.tif')
center_points = pd.read_csv('Model_components/center_points_250m_nad83.csv')

In [8]:
# resample the data onto same sized grids
interp = RegularGridInterpolator((surf_xr.y.values, surf_xr.x.values), surf_xr.values) # create an interpolator function
dem_xx = np.linspace(min(bed_xr.x.values), max(bed_xr.x.values), int(len(bed_xr.x.values))) # create an array of x values that we want to resample onto - in this case I just halfed the number of values
dem_yy = np.linspace(min(bed_xr.y.values), max(bed_xr.y.values), int(len(bed_xr.y.values)))
xx, yy = np.meshgrid(dem_xx, dem_yy) # mesh grid the 1D arrays
bb = interp((yy, xx)) # use the interpolator function on these new x and y positions

#create a new DataArray
surf_xr_resamp = xr.DataArray(bb,
coords={'y': dem_yy,'x': dem_xx}, 
dims=["y", "x"])

thick_xr = surf_xr_resamp - bed_xr

In [6]:
#create different xarrays for bed inside and outside model domain
bed_outside = bed_xr.copy()
bed_inside = bed_xr.copy()

#for bed outside, assign surface elevation within model domain so ice thickness there is 0 - this is what we solve for
xx, yy = np.meshgrid(bed_outside.x.values, bed_outside.y.values)
a = np.array([geometry.Point(x, y) for x, y in zip(xx.ravel(), yy.ravel())], dtype=object)
mask_geom1 = np.array([model_geom.contains(point) for point in a])
mask_geom1 =  mask_geom1.reshape(len(bed_outside.y), len(bed_outside.x))
bed_outside = bed_outside.where(mask_geom1 == False, surf_xr_resamp)
mask_outside = (bed_outside == bed_outside.min().values).values
bed_outside = bed_outside.where(mask_outside == False, surf_xr_resamp)

#cut down size of bed inside to be model domain size plus 1km buffer
model_shape_bounds = model_shape.total_bounds
topo_buffer = 1000
region = (model_shape_bounds[0] - topo_buffer, model_shape_bounds[2] + topo_buffer, model_shape_bounds[1] - topo_buffer, model_shape_bounds[3] + topo_buffer )
bed_inside = bed_inside.sel(y=slice(*region[2:]), x=slice(*region[:2]))
#cut down surface xarray to same area to be surf inside
surf_inside = surf_xr_resamp.copy()
surf_inside = surf_inside.sel(y=slice(*region[2:]), x=slice(*region[:2]))
#create mask around model domain
xx, yy = np.meshgrid(bed_inside.x.values, bed_inside.y.values)
a = np.array([geometry.Point(x, y) for x, y in zip(xx.ravel(), yy.ravel())], dtype=object)
mask_geom_small = np.array([model_geom.contains(point) for point in a])
mask_geom_small =  mask_geom_small.reshape(xx.shape)
bed_inside = bed_inside.where(mask_geom_small == True, surf_inside)
#create mask to exclude rock areas that may be in the model domain
mask_rock_small = (bed_inside == bed_inside.min().values).values
bed_inside = bed_inside.where(mask_rock_small == False, surf_inside)
thick_inside = surf_inside.copy() - bed_inside.copy()
#set thickness to 0 outside model domain
thick_inside = thick_inside.where(mask_geom_small == True, 0)
thick_inside = thick_inside.where(mask_rock_small == False, 0)

In [10]:
#resample dist xr to size of grid being used for inversion
interp = RegularGridInterpolator((dist_xr.y.values, dist_xr.x.values), dist_xr.values) # create an interpolator function
dem_xx = np.linspace(min(bed_inside.x.values), max(bed_inside.x.values), int(len(bed_inside.x.values))) # create an array of x values that we want to resample onto - in this case I just halfed the number of values
dem_yy = np.linspace(min(bed_inside.y.values), max(bed_inside.y.values), int(len(bed_inside.y.values)))
xx, yy = np.meshgrid(dem_xx, dem_yy) # mesh grid the 1D arrays
bb = interp((yy, xx)) # use the interpolator function on these new x and y positions

#create a new DataArray
dist_xr_resamp = xr.DataArray(bb,
coords={'y': dem_yy,'x': dem_xx}, 
dims=["y", "x"])

dist_xr_resamp = dist_xr_resamp.where(mask_geom_small == True, -99)
#set to nodata value on any intersecting rock
dist_xr_resamp = dist_xr_resamp.where(mask_rock_small == False, -99)

In [11]:
#define band_xr as distance upstream of each point

band_xr = dist_xr_resamp.copy()
all_x = xx.ravel()
all_y = yy.ravel()
for i in range(len(all_x)):
    center_points['dist_p'] = np.sqrt((center_points.xcoord - all_x[i])**2 + (center_points.ycoord - all_y[i])**2)
    dist_sorted = center_points.sort_values(by='dist_p').reset_index(drop=True)
    dist_small = dist_sorted['distance'].iloc[0]
    band_xr.loc[dict(x=all_x[i], y=all_y[i])] = dist_small

band_xr = band_xr.where(mask_geom_small == True, -900)
band_xr = band_xr.where(mask_rock_small == False, -900)

# dist_xr_buf = proxim_xr_resamp.copy()
# for i in range(len(all_x)):
#     dist_points_buf['dist_p'] = np.sqrt((dist_points_buf.xcoord - all_x[i])**2 + (dist_points_buf.ycoord - all_y[i])**2)
#     dist_sorted = dist_points_buf.sort_values(by='dist_p').reset_index(drop=True)
#     dist_small = dist_sorted['distance'].iloc[0]
#     dist_xr_buf.loc[dict(x=all_x[i], y=all_y[i])] = dist_small

# dist_xr_buf = dist_xr_buf.where(mask_geom_small == True, -900)
# dist_xr_buf = dist_xr_buf.where(mask_rock_small == False, -900)

# dist_50m_xr = proxim_xr_resamp.copy()
# for i in range(len(all_x)):
#     dist_points_50m['dist_p'] = np.sqrt((dist_points_50m.xcoord - all_x[i])**2 + (dist_points_50m.ycoord - all_y[i])**2)
#     dist_sorted = dist_points_50m.sort_values(by='dist_p').reset_index(drop=True)
#     dist_small = dist_sorted['distance'].iloc[0]
#     dist_50m_xr.loc[dict(x=all_x[i], y=all_y[i])] = dist_small

# dist_50m_xr = dist_50m_xr.where(mask_geom_small == True, -900)
# dist_50m_xr = dist_50m_xr.where(mask_rock_small == False, -900)

In [14]:
# normalise the distance from glacier edge in each band, for the bands around the curve which don't 
# extend the full distance normalise by the max value from the band and one each side
center_points['dist_max'] = -999
dist_norm_xr = dist_xr_resamp.copy()
for i in range(len(center_points)):
    band_i = center_points['distance'].iloc[i]
    if band_i > 4500 and band_i < 7500:
        max_vals = [dist_xr_resamp.where(band_xr == center_points['distance'].iloc[i]).max().values]
        for j in range(-1,2,1):
            max_vals.append(dist_xr_resamp.where(band_xr == center_points['distance'].iloc[i+j]).max().values)
        center_points['dist_max'].iloc[i] = max(max_vals)
    else:
        center_points['dist_max'].iloc[i] = dist_xr_resamp.where(band_xr == center_points['distance'].iloc[i]).max().values

    dist_norm_xr = dist_norm_xr.where(band_xr != band_i, dist_xr_resamp/center_points['dist_max'].iloc[i])

/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_23236/2528913802.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  center_points['dist_max'].iloc[i] = dist_xr_resamp.where(band_xr == center_points['distance'].iloc[i]).max().values


In [15]:
#create inside/outside array
mod_xr = dist_xr_resamp.copy()
for i in range(len(dist_xr_resamp)):
    for j in range(len(dist_xr_resamp[i])):
        #outside glacier
        if dist_xr_resamp[i][j] < 0:
            mod_xr[i][j] = 0
        #inside glacier
        else:
            mod_xr[i][j] = 1


In [19]:
#export model arrays
surf_inside.rio.write_crs(26908, inplace=True)
surf_inside.rio.to_raster('Model_components/surf_inside.tif')

bed_outside.rio.write_crs(26908, inplace=True)
bed_outside.rio.to_raster('Model_components/bed_outside.tif')

surf_xr_resamp.rio.write_crs(26908, inplace=True)
surf_xr_resamp.rio.to_raster('Model_components/surf_xr_resamp.tif')

np.savetxt('Model_components/mask_geom_small.csv', mask_geom_small, delimiter=',')
np.savetxt('Model_components/mask_rock_small.csv', mask_rock_small, delimiter=',')

band_xr.rio.write_crs(26908, inplace=True)
band_xr.rio.to_raster('Model_components/band_xr.tif')

dist_norm_xr.rio.write_crs(26908, inplace=True)
dist_norm_xr.rio.to_raster('Model_components/dist_norm_xr.tif')

mod_xr.rio.write_crs(26908, inplace=True)
mod_xr.rio.to_raster('Model_components/mod_xr.tif')